In [ ]:
import pandas as pd
from datetime import datetime
import re
import os

class LinkedInJobProcessor:
    def __init__(self):
        # Gendered words lists for analysis
        self.masculine_words = [
            'leader', 'competitive', 'dominant', 'assertive', 'aggressive', 'ambitious',
            'analytical', 'confident', 'decisive', 'determined', 'independent', 'objective',
            'self-reliant', 'strong', 'superior', 'lead', 'manage', 'direct', 'control',
            'drive', 'challenge', 'compete', 'win', 'achieve', 'dominate', 'excel',
            'individual', 'autonomous', 'hierarchy', 'decision', 'responsibility'
        ]
        
        self.feminine_words = [
            'collaborative', 'cooperative', 'supportive', 'nurturing', 'empathetic',
            'interpersonal', 'communicate', 'understand', 'responsible', 'connect',
            'honest', 'loyal', 'dependable', 'committed', 'dedicated', 'support',
            'help', 'assist', 'care', 'share', 'together', 'team', 'community',
            'relationship', 'trust', 'warm', 'kind', 'inclusive', 'collaborate'
        ]

    def clean_text(self, text):
        """Clean and normalize text"""
        if not text:
            return ""
        
        # Remove extra whitespace and normalize
        text = ' '.join(text.split())
        # Remove special characters but keep basic punctuation
        text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\(\)]', ' ', text)
        # Remove multiple spaces
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    def extract_job_id(self, url):
        """Extract job ID from LinkedIn URL"""
        try:
            # Pattern for LinkedIn URLs: /jobs/view/12345678910/
            match = re.search(r'/jobs/view/(\d+)/?', url)
            if match:
                return match.group(1)
            
            # Alternative pattern with currentJobId parameter
            match = re.search(r'currentJobId=(\d+)', url)
            if match:
                return match.group(1)
            
            # Pattern for job posting URLs
            match = re.search(r'/job-postings/(\d+)', url)
            if match:
                return match.group(1)
            
            # Fallback to last numeric part of URL
            numbers = re.findall(r'\d+', url)
            if numbers:
                return numbers[-1]
            
            return "manual_entry"
        except:
            return "manual_entry"

    def detect_remote_option(self, text):
        """Detect remote work options from text"""
        if not text:
            return "Not specified"
        
        text_lower = text.lower()
        remote_keywords = [
            'remote', 'work from home', 'telecommute', 'virtual', 'distributed',
            'home-based', 'location independent', 'anywhere', 'flexible location',
            'fully remote', '100% remote'
        ]
        hybrid_keywords = ['hybrid', 'flexible', 'part remote', 'some remote', 'flexible work']
        
        for keyword in remote_keywords:
            if keyword in text_lower:
                return "Remote"
        
        for keyword in hybrid_keywords:
            if keyword in text_lower:
                return "Hybrid"
        
        return "On-site"

    def extract_salary_info(self, text):
        """Extract salary information from text"""
        if not text:
            return "Not specified"
        
        # Look for salary patterns
        salary_patterns = [
            # Range patterns
            r'£\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*(?:-|to)\s*£?\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)',
            r'\$\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*(?:-|to)\s*\$?\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)',
            r'€\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s*(?:-|to)\s*€?\s*(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)',
            # Hourly rates
            r'(\d+(?:\.\d+)?)\s*(?:-|to)\s*(\d+(?:\.\d+)?)\s*(?:per hour|/hour|hourly)',
            # Annual salaries
            r'(\d{1,3}(?:,\d{3})*)\s*(?:-|to)\s*(\d{1,3}(?:,\d{3})*)\s*(?:per year|annually|pa|p\.a\.)',
            # Single values
            r'£(\d{1,3}(?:,\d{3})*)', r'\$(\d{1,3}(?:,\d{3})*)', r'€(\d{1,3}(?:,\d{3})*)',
            r'(\d{1,3}(?:,\d{3})*)\s*(?:per year|pa|p\.a\.)',
            # LinkedIn specific patterns
            r'(\d+(?:,\d+)?)\s*k\s*(?:-|to)\s*(\d+(?:,\d+)?)\s*k',
            r'(\d+(?:,\d+)?)\s*k\s*(?:per year|annually)'
        ]
        
        for pattern in salary_patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                if len(match.groups()) >= 2:
                    return f"{match.group(1)} - {match.group(2)}"
                else:
                    return match.group(1)
        
        return "Not specified"

    def analyze_gendered_language(self, text):
        """Analyze gendered language in text"""
        if not text:
            return 0, 0, 0, []
        
        # Clean and tokenize text
        text_lower = text.lower()
        words = re.findall(r'\b[a-z]+\b', text_lower)
        
        masculine_count = sum(1 for word in words if word in self.masculine_words)
        feminine_count = sum(1 for word in words if word in self.feminine_words)
        
        # Find gendered words present
        gendered_words_found = []
        for word in words:
            if word in self.masculine_words:
                gendered_words_found.append(f"M:{word}")
            elif word in self.feminine_words:
                gendered_words_found.append(f"F:{word}")
        
        total_words = len(words)
        if total_words == 0:
            return 0, 0, 0, gendered_words_found
        
        masculine_score = round((masculine_count / total_words) * 100, 2)
        feminine_score = round((feminine_count / total_words) * 100, 2)
        neutral_score = round(100 - masculine_score - feminine_score, 2)
        
        return masculine_score, feminine_score, neutral_score, gendered_words_found

    def determine_job_level(self, title, description):
        """Determine job level from title and description"""
        combined_text = f"{title} {description}".lower()
        
        executive_keywords = ['ceo', 'cto', 'cfo', 'coo', 'vp', 'vice president', 'president', 'executive']
        senior_keywords = ['senior', 'lead', 'principal', 'head', 'director', 'manager', 'chief', 'sr.', 'sr ']
        junior_keywords = ['junior', 'entry', 'graduate', 'intern', 'trainee', 'assistant', 'associate', 'jr.', 'jr ']
        mid_keywords = ['mid', 'intermediate', 'specialist', 'analyst', 'coordinator', 'consultant']
        
        for keyword in executive_keywords:
            if keyword in combined_text:
                return "Executive"
        
        for keyword in senior_keywords:
            if keyword in combined_text:
                return "Senior"
        
        for keyword in junior_keywords:
            if keyword in combined_text:
                return "Junior"
        
        for keyword in mid_keywords:
            if keyword in combined_text:
                return "Mid-level"
        
        return "Not specified"

    def extract_company_size(self, description):
        """Extract company size information"""
        if not description:
            return "Not specified"
        
        text_lower = description.lower()
        
        # Look for explicit employee count ranges
        size_patterns = [
            r'(\d+(?:,\d+)?)\s*(?:-|to)\s*(\d+(?:,\d+)?)\s*employees',
            r'(\d+(?:,\d+)?)\+\s*employees',
            r'over\s*(\d+(?:,\d+)?)\s*employees',
            r'more than\s*(\d+(?:,\d+)?)\s*employees'
        ]
        
        for pattern in size_patterns:
            match = re.search(pattern, text_lower)
            if match:
                return match.group(0)
        
        # Look for size descriptors
        if any(word in text_lower for word in ['startup', 'small company', 'small business']):
            return "Small (1-50)"
        elif any(word in text_lower for word in ['medium company', 'mid-size']):
            return "Medium (51-500)"
        elif any(word in text_lower for word in ['large company', 'enterprise', 'multinational', 'fortune']):
            return "Large (500+)"
        
        return "Not specified"

    def extract_industry(self, title, description, company):
        """Extract industry information"""
        combined_text = f"{title} {description} {company}".lower()
        
        # Technology
        if any(word in combined_text for word in ['software', 'technology', 'tech', 'it', 'data science', 'ai', 'machine learning']):
            return "Technology"
        
        # Finance
        if any(word in combined_text for word in ['finance', 'banking', 'financial', 'investment', 'accounting']):
            return "Finance"
        
        # Healthcare
        if any(word in combined_text for word in ['healthcare', 'medical', 'pharmaceutical', 'biotech', 'clinical']):
            return "Healthcare"
        
        # Consulting
        if any(word in combined_text for word in ['consulting', 'advisory', 'strategy', 'management consulting']):
            return "Consulting"
        
        # Marketing/Advertising
        if any(word in combined_text for word in ['marketing', 'advertising', 'digital marketing', 'brand', 'social media']):
            return "Marketing/Advertising"
        
        # Education
        if any(word in combined_text for word in ['education', 'university', 'school', 'training', 'learning']):
            return "Education"
        
        # Manufacturing
        if any(word in combined_text for word in ['manufacturing', 'production', 'industrial', 'factory']):
            return "Manufacturing"
        
        # Retail
        if any(word in combined_text for word in ['retail', 'e-commerce', 'consumer', 'sales']):
            return "Retail"
        
        return "Not specified"

    def get_job_input(self, url):
        """Get job information through user input"""
        print("\n" + "="*60)
        print("JOB DATA ENTRY - LINKEDIN")
        print("="*60)
        print(f"URL: {url}")
        print("-"*60)
        
        # Get basic information
        job_title = input("Job Title: ").strip()
        company_name = input("Company Name: ").strip()
        location = input("Location: ").strip()
        
        # Get LinkedIn-specific information
        posting_date = input("Posting Date (optional): ").strip()
        employment_type = input("Employment Type (e.g., Full-time, Part-time, Contract) (optional): ").strip()
        experience_level = input("Experience Level (e.g., Entry level, Mid-senior) (optional): ").strip()
        workplace_type = input("Workplace Type (e.g., On-site, Remote, Hybrid) (optional): ").strip()
        salary_info = input("Salary Info (optional): ").strip()
        
        # Get job description
        print("\nJob Description:")
        print("(Paste the job description below. Press Enter twice when finished)")
        print("-" * 40)
        
        description_lines = []
        empty_line_count = 0
        
        while empty_line_count < 2:
            try:
                line = input()
                if line.strip() == "":
                    empty_line_count += 1
                else:
                    empty_line_count = 0
                description_lines.append(line)
            except EOFError:
                break
        
        description = '\n'.join(description_lines).strip()
        
        return {
            'job_title': job_title,
            'company_name': company_name,
            'location': location,
            'posting_date': posting_date,
            'employment_type': employment_type,
            'experience_level': experience_level,
            'workplace_type': workplace_type,
            'salary_info': salary_info,
            'description': description
        }

    def process_job_data(self, input_data, url):
        """Process the manually entered job data"""
        
        # Clean the description
        description_cleaned = self.clean_text(input_data['description'])
        
        # Create job data structure
        job_data = {
            'job_id': self.extract_job_id(url),
            'platform': 'LinkedIn',
            'job_url': url,
            'scrape_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'job_title': input_data['job_title'],
            'company_name': input_data['company_name'],
            'location': input_data['location'],
            'remote_option': '',
            'posting_date': input_data['posting_date'] or 'Not specified',
            'employment_type': input_data['employment_type'] or 'Not specified',
            'experience_level': input_data['experience_level'] or 'Not specified',
            'workplace_type': input_data['workplace_type'] or 'Not specified',
            'job_description_raw': input_data['description'],
            'job_description_cleaned': description_cleaned,
            'salary_info': '',
            'industry': '',
            'company_size': '',
            'job_level': '',
            'gendered_words_found': '',
            'masculine_score': 0,
            'feminine_score': 0,
            'neutral_score': 0,
            'seniority_flag': '',
            'word_count': 0,
            'manual_review_flag': False,
            'ad_url': url
        }
        
        # Process salary info
        if input_data['salary_info']:
            job_data['salary_info'] = input_data['salary_info']
        else:
            job_data['salary_info'] = self.extract_salary_info(description_cleaned)
        
        # Determine remote option (use workplace_type if provided, otherwise analyze text)
        if input_data['workplace_type']:
            job_data['remote_option'] = input_data['workplace_type']
        else:
            combined_text = f"{job_data['job_title']} {job_data['location']} {description_cleaned}"
            job_data['remote_option'] = self.detect_remote_option(combined_text)
        
        # Determine job level
        job_data['job_level'] = self.determine_job_level(job_data['job_title'], description_cleaned)
        job_data['seniority_flag'] = job_data['job_level'] in ["Senior", "Executive"]
        
        # Determine industry
        job_data['industry'] = self.extract_industry(
            job_data['job_title'], description_cleaned, job_data['company_name']
        )
        
        # Extract company size
        job_data['company_size'] = self.extract_company_size(description_cleaned)
        
        # Analyze gendered language
        masculine_score, feminine_score, neutral_score, gendered_words = self.analyze_gendered_language(description_cleaned)
        
        job_data['masculine_score'] = masculine_score
        job_data['feminine_score'] = feminine_score
        job_data['neutral_score'] = neutral_score
        job_data['gendered_words_found'] = '; '.join(gendered_words) if gendered_words else ''
        
        # Word count
        if description_cleaned:
            job_data['word_count'] = len(description_cleaned.split())
        
        # Manual review flag
        if (job_data['word_count'] < 20 or 
            not job_data['job_title'] or 
            not job_data['company_name']):
            job_data['manual_review_flag'] = True
        
        return job_data

    def save_to_excel(self, job_data, output_path, append_mode=False):
        """Save job data to Excel file"""
        try:
            # Ensure output directory exists
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # Create DataFrame
            df_new = pd.DataFrame([job_data])
            
            if append_mode and os.path.exists(output_path):
                # Read existing data and append
                try:
                    df_existing = pd.read_excel(output_path)
                    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
                except:
                    df_combined = df_new
            else:
                df_combined = df_new
            
            # Save to Excel
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                df_combined.to_excel(writer, sheet_name='Job_Data', index=False)
            
            print(f"\nData saved to: {output_path}")
            print(f"Total jobs in file: {len(df_combined)}")
            return True
            
        except Exception as e:
            print(f"Error saving to Excel: {e}")
            return False

def main():
    """Main function for manual job data entry"""
    
    # Configuration
    OUTPUT_PATH = r"C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Codes\linkedin_jobs_data.xlsx"
    
    print("="*70)
    print("LINKEDIN JOB DATA PROCESSOR - MANUAL ENTRY")
    print("="*70)
    print("This tool helps you manually enter job data from LinkedIn and performs analysis.")
    print("You can process multiple jobs - the data will be appended to the Excel file.")
    print("="*70)
    
    processor = LinkedInJobProcessor()
    
    while True:
        print("\n" + "="*40)
        print("NEW JOB ENTRY")
        print("="*40)
        
        # Get job URL
        url = input("Enter LinkedIn job URL: ").strip()
        if not url:
            print("No URL provided. Exiting...")
            break
        
        # Get job data
        input_data = processor.get_job_input(url)
        
        # Process the data
        job_data = processor.process_job_data(input_data, url)
        
        # Display results
        print("\n" + "="*60)
        print("PROCESSING RESULTS")
        print("="*60)
        print(f"Job Title: {job_data['job_title']}")
        print(f"Company: {job_data['company_name']}")
        print(f"Location: {job_data['location']}")
        print(f"Remote Option: {job_data['remote_option']}")
        print(f"Employment Type: {job_data['employment_type']}")
        print(f"Experience Level: {job_data['experience_level']}")
        print(f"Job Level: {job_data['job_level']}")
        print(f"Industry: {job_data['industry']}")
        print(f"Company Size: {job_data['company_size']}")
        print(f"Salary Info: {job_data['salary_info']}")
        print(f"Posting Date: {job_data['posting_date']}")
        print(f"Word Count: {job_data['word_count']}")
        print(f"Masculine Score: {job_data['masculine_score']}%")
        print(f"Feminine Score: {job_data['feminine_score']}%")
        print(f"Neutral Score: {job_data['neutral_score']}%")
        if job_data['gendered_words_found']:
            print(f"Gendered Words: {job_data['gendered_words_found']}")
        print(f"Manual Review Flag: {job_data['manual_review_flag']}")
        
        # Save to Excel
        append_mode = os.path.exists(OUTPUT_PATH)
        success = processor.save_to_excel(job_data, OUTPUT_PATH, append_mode)
        
        if success:
            print("\n✅ Job data saved successfully!")
        else:
            print("\n❌ Failed to save job data.")
        
        # Ask if user wants to add another job
        print("\n" + "-"*40)
        continue_choice = input("Do you want to add another job? (y/n): ").strip().lower()
        if continue_choice not in ['y', 'yes']:
            break
    
    print("\n" + "="*70)
    print("SESSION COMPLETED")
    print("="*70)
    print(f"All job data has been saved to: {OUTPUT_PATH}")
    print("Thank you for using the LinkedIn Job Data Processor!")

if __name__ == "__main__":
    main()

LINKEDIN JOB DATA PROCESSOR - MANUAL ENTRY
This tool helps you manually enter job data from LinkedIn and performs analysis.
You can process multiple jobs - the data will be appended to the Excel file.

NEW JOB ENTRY


Enter LinkedIn job URL:  https://www.linkedin.com/jobs/search-results/?currentJobId=4262223897&eBP=NON_CHARGEABLE_CHANNEL&geoId=101165590&keywords=operational%20research&origin=JOBS_HOME_SEARCH_BUTTON&refId=0bW7n%2Fc0HG5xoT8YXWFjJw%3D%3D&trackingId=98rKMGkSSU0LYDYhX2Yhlg%3D%3D



JOB DATA ENTRY - LINKEDIN
URL: https://www.linkedin.com/jobs/search-results/?currentJobId=4262223897&eBP=NON_CHARGEABLE_CHANNEL&geoId=101165590&keywords=operational%20research&origin=JOBS_HOME_SEARCH_BUTTON&refId=0bW7n%2Fc0HG5xoT8YXWFjJw%3D%3D&trackingId=98rKMGkSSU0LYDYhX2Yhlg%3D%3D
------------------------------------------------------------


Job Title:  Data Science Manager - Operational Research
Company Name:   KDR Talent Solutions
Location:  London
Posting Date (optional):  
Employment Type (e.g., Full-time, Part-time, Contract) (optional):  
Experience Level (e.g., Entry level, Mid-senior) (optional):  
Workplace Type (e.g., On-site, Remote, Hybrid) (optional):  
Salary Info (optional):  



Job Description:
(Paste the job description below. Press Enter twice when finished)
----------------------------------------


...


Job Description:
(Paste the job description below. Press Enter twice when finished)
----------------------------------------


 The Ellison Institute of Technology (EIT) works to develop and deploy advanced technology in pursuit of solving some of humanity's most challenging and enduring problems. Guided by world-leading scientists and entrepreneurs, EIT seeks to accelerate innovation by driving scientific and technological advancements across four humane endeavours:health and medical science, food security and sustainable agriculture, climate change and clean energy, and government innovation and the era of artificial intelligence.  Please visit eit.org for more details.  Our Culture  At the Ellison Institute, we believe that an inclusive, collaborative team atmosphere is just as important to our mission as our scientific aims and methods.  We strive to build a supportive environment where everyone feels confident taking creative risks toward innovation. We value emotional intelligence and communicating with empathy and respect for others.  We foster a team that is curious, has a deep sense of commitment, res


PROCESSING RESULTS
Job Title: AI Research and Robotics Manager
Company: Ellison Institute of Technology Oxford
Location: Oxford
Remote Option: On-site
Employment Type: Not specified
Experience Level: Not specified
Job Level: Executive
Industry: Technology
Company Size: Not specified
Salary Info: Not specified
Posting Date: Not specified
Word Count: 743
Masculine Score: 1.71%
Feminine Score: 3.15%
Neutral Score: 95.14%
Gendered Words: F:inclusive; F:collaborative; F:team; F:supportive; M:confident; F:team; M:responsibility; M:achieve; F:team; M:lead; F:collaborative; M:lead; F:team; M:strong; F:team; F:collaborative; F:team; F:community; M:lead; F:community; M:manage; M:drive; F:support; F:help; F:team; F:support; F:support; M:decision; M:drive; M:confident; F:community; F:trust; M:strong; F:collaborative; F:inclusive; F:team; F:supportive
Manual Review Flag: False

Data saved to: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Codes\linke

Do you want to add another job? (y/n):  y



NEW JOB ENTRY
